In [2]:
import category_encoders as ce
import pandas as pd
import numpy as np

final_cars = pd.read_csv("./data/processed_cars.csv")
final_cars = final_cars.drop(columns=["name", "source_url"])


In [3]:
type1 = final_cars['type']
type1 = pd.get_dummies(type1,drop_first=True)
type1.head()


,coupe,crossover,hatchback,pickup,sedan,suv,van,wagon
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0


In [4]:
transmission = final_cars['transmission']
transmission = pd.get_dummies(transmission,drop_first=True)
transmission.head()

,manual
0,0
1,0
2,0
3,0
4,0


In [5]:
fuels = final_cars['fuels']
fuels = pd.get_dummies(fuels,drop_first=True)
fuels.head()

,electric,gasoline,hybrid
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


In [6]:
origin = final_cars['origin']
origin = pd.get_dummies(origin,drop_first=True)
origin.head()

,imported
0,1
1,1
2,1
3,1
4,1


In [7]:
final_cars = pd.concat([final_cars, transmission, fuels, origin, type1],axis = 1)
drop_cars = final_cars.drop(columns=["type", "origin", "external_color", "fuels", "transmission", "wheel_drive", "internal_color"])

drop_cars

,Unnamed: 0,Unnamed: 0.1,brand,km_driven,seats,engine_capacity,price,year,model,manual,...,hybrid,imported,coupe,crossover,hatchback,pickup,sedan,suv,van,wagon
0,0,0.0,acura,0.0,5,3.7,1280000000,2010,ZDX,0,...,0,1,0,1,0,0,0,0,0,0
1,1,1.0,acura,0.0,5,3.7,600000000,2008,MDX,0,...,0,1,0,0,0,0,0,1,0,0
2,2,2.0,acura,110000.0,5,3.7,960000000,2010,ZDX,0,...,0,1,0,1,0,0,0,0,0,0
3,3,3.0,acura,0.0,5,3.7,650000000,2007,MDX,0,...,0,1,0,0,0,0,0,1,0,0
4,4,4.0,acura,10000.0,5,3.7,590000000,2009,MDX,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28950,28952,28952.0,volvo,0.0,5,2.0,1580000000,2016,XC60,0,...,0,1,0,0,0,0,0,1,0,0
28951,28953,28953.0,volvo,6000.0,5,2.0,3250000000,2017,XC90,0,...,0,1,0,0,0,0,0,1,0,0
28952,28954,28954.0,zotye,100000.0,5,2.0,500000000,2019,Z8,0,...,0,1,0,0,0,0,0,1,0,0
28953,28955,28955.0,zotye,38000.0,5,2.0,495000000,2019,Z8,0,...,0,1,0,0,0,0,0,1,0,0


In [8]:
drop_cars["brand"] = ce.JamesSteinEncoder().fit_transform(drop_cars["brand"], drop_cars["price"])
drop_cars["model"] = ce.JamesSteinEncoder().fit_transform(drop_cars["model"], drop_cars["price"])


encoder
print(drop_cars["brand"])

0        9.452979e+08
1        9.452979e+08
2        9.452979e+08
3        9.452979e+08
4        9.452979e+08
             ...     
28950    2.501147e+09
28951    2.501147e+09
28952    4.867135e+08
28953    4.867135e+08
28954    4.867135e+08
Name: brand, Length: 28955, dtype: float64


In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

X = drop_cars.drop(["price"], axis=1)
y = drop_cars["price"]

VERBOSE = 0
SEED = 42


X_train , X_test, y_train, y_test = train_test_split(X, y, random_state = SEED , test_size=0.2)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)


[[-0.24736293 -0.24736293 -0.65687512 ... -0.62675405 -0.22751201
  -0.01609626]
 [ 1.10505828  1.10505828 -0.21997028 ...  1.59552221 -0.22751201
  -0.01609626]
 [ 0.16011488  0.16011488  1.32990148 ...  1.59552221 -0.22751201
  -0.01609626]
 ...
 [-1.62492385 -1.62492385 -0.90111539 ... -0.62675405 -0.22751201
  -0.01609626]
 [ 0.15451503  0.15451503  1.32990148 ... -0.62675405 -0.22751201
  -0.01609626]
 [ 1.09087996  1.09087996 -0.21997028 ... -0.62675405 -0.22751201
  -0.01609626]]


In [10]:

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

regressors = [
    LinearRegression(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
]

res = pd.DataFrame(columns=['Model', 'RMSE', 'R-Squared'])
RMSE = []
R2 = []
for num, regressor in enumerate(regressors):
    reg = regressor.fit(X_train, y_train)
    pred = reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(pred, y_test))
    r2 = r2_score(pred, y_test)
    RMSE.append(str(np.around(rmse)))
    R2.append(str(round(r2, 2)*100) + '%')
res['Model'] = ['LinearRegression', 'RandomForestRegressor', 'DecisionTreeRegressor']
res['RMSE'] = RMSE
res['R-Squared'] = R2
res

,Model,RMSE,R-Squared
0,LinearRegression,1331426395.0,32.0%
1,RandomForestRegressor,435492381.0,96.0%
2,DecisionTreeRegressor,465887899.0,95.0%
